In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
from bokeh import events
from bokeh.io import output_file, output_notebook, curdoc
from bokeh.plotting import figure, show
from bokeh.models import CustomJS, ColumnDataSource, LinearAxis, Range1d, Toggle, Slider, DatetimeTickFormatter, Select, Tabs, TabPanel, HoverTool, CrosshairTool, Span, Div, RadioGroup
from bokeh.layouts import row, column, gridplot, layout
from bokeh_realtime_packagev1.functions import *
from bokeh_realtime_packagev1.conditions import *
output_file('layout3_v003.html')  # Render to static HTML, or
output_notebook()  # Render inline in a Jupyter Notebook
from icecream import ic
from iertools.tb import TB
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns',None)

from dateutil.parser import parse
from scipy.stats import norm

Loading BokehJS ...

## Creating water consume data

In [2]:
def water_consume_normal(fechas,param):
    x = np.linspace(norm.ppf(0.01),norm.ppf(0.99),288)
    y = norm.pdf(x)+np.random.uniform(0,0.2,288)
    for i in range(len(fechas)):
        if fechas[i].hour < 6:
            y[i] = 0
        elif fechas[i].hour >= 21:
            y[i] = 0
    consume=y*param
    return consume
def electric_consume_normal(param):
    x = np.linspace(norm.ppf(0.01),norm.ppf(0.99),288)
    y = norm.pdf(x)+np.random.uniform(0,0.2,288)
    consume=y*param
    return consume

In [6]:
fechas=[]
for i in range(287):
    fechas.append(parse('2019-01-01 00:05:00')+(pd.Timedelta('5min')*i))
fechas.append(parse('2019-01-01 23:59:59'))
water_zones = ['PBBAÑOH','PBBAÑOM','N1BAÑOH','N1BAÑOM','N2BAÑOH','N2BAÑOM','N1LAB','N2LAB1','N2LAB2','PBCAFE','PBBEB1','PBBEB2','N1BEB','N2BEB']
data = pd.DataFrame()
for zone in water_zones:
    data['water_'+zone] = water_consume_normal(fechas,500)

tfechas=[]
for i in range(287):
    tfechas.append(parse('2019-01-02 00:05:00')+(pd.Timedelta('5min')*i))
    fechas.append(parse('2019-01-02 00:05:00')+(pd.Timedelta('5min')*i))
fechas.append(parse('2019-01-02 23:59:59'))
tfechas.append(parse('2019-01-02 23:59:59'))
water_zones = ['PBBAÑOH','PBBAÑOM','N1BAÑOH','N1BAÑOM','N2BAÑOH','N2BAÑOM','N1LAB','N2LAB1','N2LAB2','PBCAFE','PBBEB1','PBBEB2','N1BEB','N2BEB']
tmp = pd.DataFrame()
for zone in water_zones:
    tmp['water_'+zone] = water_consume_normal(tfechas,500)
data = pd.concat([data,tmp],ignore_index=True)

tfechas=[]
for i in range(287):
    tfechas.append(parse('2019-01-03 00:05:00')+(pd.Timedelta('5min')*i))
    fechas.append(parse('2019-01-03 00:05:00')+(pd.Timedelta('5min')*i))
tfechas.append(parse('2019-01-03 23:59:59'))
fechas.append(parse('2019-01-03 23:59:59'))
water_zones = ['PBBAÑOH','PBBAÑOM','N1BAÑOH','N1BAÑOM','N2BAÑOH','N2BAÑOM','N1LAB','N2LAB1','N2LAB2','PBCAFE','PBBEB1','PBBEB2','N1BEB','N2BEB']
tmp = pd.DataFrame()
for zone in water_zones:
    tmp['water_'+zone] = water_consume_normal(tfechas,500)
data = pd.concat([data,tmp],ignore_index=True)
data['ts']=fechas
data['time']=fechas
# data.set_index('ts',inplace=True)
# data

In [ ]:
data

## Creating electric dummy consume data

In [7]:
electric_zones = ['N1AU401', 'N1AU402', 'N1AU403', 'N1AU404', 'N2AU101', 'N2AU102', 'N2AU103', 'N2AU201', 'N2AU202', 'N2AU203', 'PBADM', 'PBCOORCOFI', 'PBCOORDINACION', 'PBSJUNTAS', 'PBSMAESTROS', 'PBVESTIBULO', 'N1LAB', 'N2LAB1', 'N2LAB2', 'PBCAFE']
tmp=pd.DataFrame()
for zone in electric_zones:
    tmp['elec_c_'+zone]=electric_consume_normal(200)

tmp2=pd.DataFrame()
for zone in electric_zones:
    tmp2['elec_c_'+zone]=electric_consume_normal(200)
tmp = pd.concat([tmp,tmp2],ignore_index=True)
tmp2=pd.DataFrame()
for zone in electric_zones:
    tmp2['elec_c_'+zone]=electric_consume_normal(200)
tmp = pd.concat([tmp,tmp2],ignore_index=True)
data = pd.concat([data,tmp],axis=1)

tmp=pd.DataFrame()
for zone in electric_zones:
    tmp['elec_l_'+zone]=electric_consume_normal(20)

tmp2=pd.DataFrame()
for zone in electric_zones:
    tmp2['elec_l_'+zone]=electric_consume_normal(20)
tmp = pd.concat([tmp,tmp2],ignore_index=True)
tmp2=pd.DataFrame()
for zone in electric_zones:
    tmp2['elec_l_'+zone]=electric_consume_normal(20)
tmp = pd.concat([tmp,tmp2],ignore_index=True)
data = pd.concat([data,tmp],axis=1)
data.set_index('ts',inplace=True)
# data

In [10]:
data.to_csv('../data/comsuption_data.csv')

## Plotting

In [6]:
baños=[]
labs=[]
cafe=[]
beb=[]
for zone in water_zones:
    if 'BAÑO' in zone:
        baños.append('water_'+zone)
    elif 'LAB' in zone:
        labs.append('water_'+zone)
    elif 'CAFE' in zone:
        cafe.append('water_'+zone)
    elif 'BEB' in zone:
        beb.append('water_'+zone)

data['WBAÑOS']=data[baños].sum(axis=1)
data['WLABS']=data[labs].sum(axis=1)
data['WCAFE']=data[cafe].sum(axis=1)
data['WBEB']=data[beb].sum(axis=1)

In [7]:
cont=[]
labs=[]
ofi=[]
for zone in electric_zones:
    if 'AU' in zone:
        cont.append('elec_c_'+zone)
    elif 'LAB' in zone:
        labs.append('elec_c_'+zone)
    elif 'PB' in zone:
        ofi.append('elec_c_'+zone)

data['CONTACTOSAU']=data[cont].sum(axis=1)
data['EQUIPOLAB']=data[labs].sum(axis=1)
data['EQUIPOOFI']=data[ofi].sum(axis=1)
data['LUCES']=data[['elec_l_'+zone for zone in electric_zones]].sum(axis=1)

In [34]:
data.iloc[0:288]

,water_PBBAÑOH,water_PBBAÑOM,water_N1BAÑOH,water_N1BAÑOM,water_N2BAÑOH,water_N2BAÑOM,water_N1LAB,water_N2LAB1,water_N2LAB2,water_PBCAFE,water_PBBEB1,water_PBBEB2,water_N1BEB,water_N2BEB,time,elec_c_N1AU401,elec_c_N1AU402,elec_c_N1AU403,elec_c_N1AU404,elec_c_N2AU101,elec_c_N2AU102,elec_c_N2AU103,elec_c_N2AU201,elec_c_N2AU202,elec_c_N2AU203,elec_c_PBADM,elec_c_PBCOORCOFI,elec_c_PBCOORDINACION,elec_c_PBSJUNTAS,elec_c_PBSMAESTROS,elec_c_PBVESTIBULO,elec_c_N1LAB,elec_c_N2LAB1,elec_c_N2LAB2,elec_c_PBCAFE,elec_l_N1AU401,elec_l_N1AU402,elec_l_N1AU403,elec_l_N1AU404,elec_l_N2AU101,elec_l_N2AU102,elec_l_N2AU103,elec_l_N2AU201,elec_l_N2AU202,elec_l_N2AU203,elec_l_PBADM,elec_l_PBCOORCOFI,elec_l_PBCOORDINACION,elec_l_PBSJUNTAS,elec_l_PBSMAESTROS,elec_l_PBVESTIBULO,elec_l_N1LAB,elec_l_N2LAB1,elec_l_N2LAB2,elec_l_PBCAFE,WBAÑOS,WLABS,WCAFE,WBEB,CONTACTOSAU,EQUIPOLAB,EQUIPOOFI,LUCES
ts,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:05:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-01-01 00:05:00,22.276969,23.407358,21.037311,36.188187,39.909377,20.391450,36.055991,42.799279,22.736923,30.418731,36.297427,17.031259,20.344733,32.468786,30.221553,16.421099,14.104492,40.564560,10.505530,31.758330,1.331079,2.267122,0.905579,3.793813,0.670947,2.011381,1.880663,0.839543,3.523632,3.156230,4.098544,1.853065,1.701737,2.672160,2.393584,2.954011,3.047148,0.999027,4.068381,4.298397,0.0,0.0,0.0,0.0,295.221575,65.174582,184.543186,48.466044
2019-01-01 00:10:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-01-01 00:10:00,28.003760,30.684116,19.778501,24.792108,18.225061,17.840748,25.223548,14.369221,24.424211,6.581276,6.357503,10.155596,15.426394,17.193861,28.598263,17.476654,40.801331,22.634394,10.423483,17.215275,3.975003,4.020715,0.758833,2.995193,4.010626,2.698799,2.669188,4.027441,2.719860,4.239243,0.708465,3.539603,2.717207,1.444411,3.482885,2.215997,3.076910,0.764627,2.996109,1.259347,0.0,0.0,0.0,0.0,209.922548,73.859208,112.423547,54.320460
2019-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-01-01 00:15:00,39.688824,40.973324,9.133909,45.533032,11.788334,20.768311,26.758131,45.541443,41.017003,21.153169,8.422810,40.103457,40.755498,27.113609,40.665965,9.738102,26.534628,44.196843,23.598736,23.618525,2.260683,1.932516,3.933159,1.170358,2.052105,4.106500,2.444721,3.701344,1.964869,0.586820,1.579846,4.085776,1.444364,1.845908,3.938255,0.947743,0.899994,1.426671,2.703441,3.381536,0.0,0.0,0.0,0.0,302.355481,94.330207,190.417965,46.406609
2019-01-01 00:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-01-01 00:20:00,16.490200,27.197725,40.801400,42.261382,34.218738,8.282682,38.493888,37.117505,33.452508,16.204986,7.780891,41.323729,19.504727,7.929718,34.671245,26.570815,32.691785,24.524403,13.043557,10.991545,0.854317,1.273180,2.860587,0.956834,4.234141,1.627284,4.338127,3.521067,3.921951,3.847438,2.036011,3.411277,0.947925,2.268358,1.569549,3.674252,3.771497,2.550135,4.151103,0.988333,0.0,0.0,0.0,0.0,294.521014,70.259746,148.772671,52.803364
2019-01-01 00:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-01-01 00:25:00,24.925658,27.075083,26.584932,24.682502,28.568532,41.903478,20.302491,26.099379,35.132066,34.649556,15.490691,12.963334,12.510931,40.737541,21.373511,32.549297,30.824384,37.283638,43.305360,44.077277,1.901165,3.567723,2.032502,4.551818,1.002698,1.837716,4.402965,3.743307,3.834095,1.387664,0.776969,2.371832,4.136020,2.297017,2.659255,3.406561,1.289760,2.532309,4.416831,2.667679,0.0,0.0,0.0,0.0,289.923676,111.413382,179.702583,54.815885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-01 23:40:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-01-01 23:40:00,42.011328,30.686353,44.402453,31.855010,

In [75]:
wzones=['Baños','Laboratorios','Cafetería','Bebederos']
water_accumulate=pd.DataFrame(index=wzones)
for i in range(3):
    extract=data.iloc[0+288*i:288+288*i]
    baños=[]
    labs=[]
    cafe=[]
    beb=[]
    for zone in water_zones:
        if 'BAÑO' in zone:
            baños.append(extract['water_'+zone].sum())
        elif 'LAB' in zone:
            labs.append(extract['water_'+zone].sum())
        elif 'CAFE' in zone:
            cafe.append(extract['water_'+zone].sum())
        elif 'BEB' in zone:
            beb.append(extract['water_'+zone].sum())
    water_accumulate['dia'+str(i)]=[sum(baños),sum(labs),sum(cafe),sum(beb)]
water_accumulate.rename(columns={'dia0':'Antier','dia1':'Ayer','dia2':'Hoy'},inplace=True)
water_accumulate = water_accumulate.T

In [49]:
ezones=['Contactos Aulas','Equipo Laboratorio','Equipo de Oficina','Luces']
electric_accumulate=pd.DataFrame(index=ezones,in)
for i in range(3):
    extract=data.iloc[0+288*i:288+288*i]
    cont=[]
    labs=[]
    ofi=[]
    light=[]
    for zone in data.columns:
        if 'elec_c_' in zone:
            if 'AU' in zone:
                cont.append(extract[zone].sum())
            elif 'LAB' in zone:
                labs.append(extract[zone].sum())
            elif 'PB' in zone:
                ofi.append(extract[zone].sum())
        elif 'elec_l_' in zone:
            light.append(extract[zone].sum())
    electric_accumulate['dia'+str(i)]=[sum(cont),sum(labs),sum(ofi),sum(light)]
electric_accumulate.rename(columns={'dia0':'Antier','dia1':'Ayer','dia2':'Hoy'},inplace=True)
electric_accumulate = electric_accumulate.T

In [80]:
dat = ColumnDataSource(data.iloc[0:288])
p = figure(height=530,width=960,toolbar_location=None,tooltips='',x_axis_type='datetime')

p.varea_stack(['WBAÑOS','WLABS','WCAFE','WBEB'],x='ts',source=dat,fill_color=['aqua','darkcyan','slateblue','mediumblue'],legend_label=['Baños','Laboratorios','Cafetería','Bebederos'])

p2 = figure(height=530,width=960,toolbar_location=None,tooltips='',x_axis_type='datetime')

p2.varea_stack(['CONTACTOSAU','EQUIPOLAB','EQUIPOOFI','LUCES'],x='ts',source=dat,fill_color=['darkorange','aquamarine','sienna','gold'],legend_label=['Aulas', 'Laboratorios','Uso general','Iluminación'])

awater= ColumnDataSource(water_accumulate)
p3 = figure(height=530,width=960,toolbar_location=None,tooltips='',x_range=['Antier','Ayer','Hoy'])

p3.line(x='index',y=wzones[0],source=awater,legend_label=wzones[0],color='aqua',line_width=5)
p3.circle(x='index',y=wzones[0],source=awater,fill_color='aqua',line_color='black',size=10)

p3.line(x='index',y=wzones[1],source=awater,legend_label=wzones[1],color='darkcyan',line_width=5)
p3.circle(x='index',y=wzones[1],source=awater,fill_color='darkcyan',line_color='black',size=10)

p3.line(x='index',y=wzones[2],source=awater,legend_label=wzones[2],color='slateblue',line_width=5)
p3.circle(x='index',y=wzones[2],source=awater,fill_color='slateblue',line_color='black',size=10)

p3.line(x='index',y=wzones[3],source=awater,legend_label=wzones[3],color='mediumblue',line_width=5)
p3.circle(x='index',y=wzones[3],source=awater,fill_color='mediumblue',line_color='black',size=10)

aelectric= ColumnDataSource(electric_accumulate)
p4 = figure(height=530,width=960,toolbar_location=None,tooltips='',x_range=['Antier','Ayer','Hoy'])

p4.line(x='index',y=ezones[0],source=aelectric,legend_label=ezones[0],color='darkorange',line_width=5)
p4.circle(x='index',y=ezones[0],source=aelectric,fill_color='darkorange',line_color='black',size=10)

p4.line(x='index',y=ezones[1],source=aelectric,legend_label=ezones[1],color='aquamarine',line_width=5)
p4.circle(x='index',y=ezones[1],source=aelectric,fill_color='aquamarine',line_color='black',size=10)

p4.line(x='index',y=ezones[2],source=aelectric,legend_label=ezones[2],color='sienna',line_width=5)
p4.circle(x='index',y=ezones[2],source=aelectric,fill_color='sienna',line_color='black',size=10)

p4.line(x='index',y=ezones[3],source=aelectric,legend_label=ezones[3],color='gold',line_width=5)
p4.circle(x='index',y=ezones[3],source=aelectric,fill_color='gold',line_color='black',size=10)



l3=layout([[p,p2],[p3,p4]])
show(l3)